<a href="https://www.kaggle.com/code/mh0386/medical-insurance-cost-prediction?scriptVersionId=120955316" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# The Data

In [2]:
data = pd.read_csv('/kaggle/input/insurance/insurance.csv')
data

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


# Null Values

In [3]:
data.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
charges     0
dtype: int64

# Y_train

In [4]:
Y_train = data.charges
data.drop('charges', axis=1, inplace=True)
Y_train

0       16884.92400
1        1725.55230
2        4449.46200
3       21984.47061
4        3866.85520
           ...     
1333    10600.54830
1334     2205.98080
1335     1629.83350
1336     2007.94500
1337    29141.36030
Name: charges, Length: 1338, dtype: float64

# Feature Scaling 

In [5]:
data.region.unique()

array(['southwest', 'southeast', 'northwest', 'northeast'], dtype=object)

In [6]:
data['sex'].replace(['female','male'],[0,1], inplace=True)
data['smoker'].replace(['no','yes'],[0,1], inplace=True)
data['region'].replace(['southwest', 'southeast', 'northwest', 'northeast'],[0,1,10,11], inplace=True)
data

,age,sex,bmi,children,smoker,region
0,19,0,27.900,0,1,0
1,18,1,33.770,1,0,1
2,28,1,33.000,3,0,1
3,33,1,22.705,0,0,10
4,32,1,28.880,0,0,10
...,...,...,...,...,...,...
1333,50,1,30.970,3,0,10
1334,18,0,31.920,0,0,11
1335,18,0,36.850,0,0,1
1336,21,0,25.800,0,0,0


# Predict Function 

In [7]:
def predict(x, w, b): 
    m = x.shape[0]
    p = np.array([])
    for i in range(m):
        p = np.append(p,np.dot(x[i], w) + b)
    return p

# Cost Function

In [8]:
def cost(x, w, b, Y_train):
    m = x.shape[0]
    cost = 0
    loss = np.zeros(m)
    for i in range(m):
        pred = np.dot(x[i], w) + b
        loss[i] = ((pred-Y_train[i])**2)/2
    cost = np.sum(loss)/m
    return cost, loss

# Train The Model

In [9]:
X_train = np.asmatrix(data)
b_init = np.random.rand(1)
w_init = np.random.rand(6)
predicted = predict(X_train,w_init,b_init)
predicted

array([31.06979986, 36.93276359, 40.43385067, ..., 38.58257335,
       28.85060552, 51.08230791])

# Calculate Model Cost

In [10]:
costVal, lossVal = cost(X_train,w_init,b_init,Y_train)
costVal
lossVal

array([1.42026201e+08, 1.42571797e+06, 9.71976461e+06, ...,
       1.26603976e+06, 1.95840731e+06, 4.23122137e+08])